Подключаем библиотеки

In [3]:
from nornir_netbox.plugins.inventory.netbox import NetBoxInventory2
from nornir import InitNornir
from nornir_netmiko.tasks import netmiko_send_command, netmiko_send_config
from nornir_jinja2.plugins.tasks import template_file
from nornir_utils.plugins.functions import print_result
from nornir.core.task import Task, Result
import pynetbox
from nornir_napalm.plugins.tasks import napalm_get
from nornir_napalm.plugins.tasks import napalm_cli
from nornir.core.filter import F
#from nornir.core.inventory import Host
import json
import pprint
from ipaddress import IPv4Address
from ipaddress import IPv4Network
from jinja2 import Environment, FileSystemLoader

Инициализируем nornir

In [4]:
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.101:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

Инициализируем pynetbox

In [5]:
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [6]:
#list(nb.dcim.devices.all())

GET ALL DEVICES PER SITE

In [7]:
nb_site = 'plgn'
#nb_role = 'plgn'
get_all_devices = list(nb.dcim.devices.filter(site = nb_site))
#get_all_devices

GET ALL IP PREFIXES PER SITE

In [8]:
# List all IPs prefixes from all sites
nb_site = 'plgn'
nb_role = 'plgn'
IPv4 = 4
get_all_ip_prefixes = list(nb.ipam.prefixes.filter(site = nb_site, role = nb_role))
for ip_prefix in get_all_ip_prefixes:
    if ip_prefix.family.value == IPv4:
        print(ip_prefix.description)
        print(ip_prefix)

Servers pool
10.30.1.0/24
Servers test pool
10.30.3.0/24


GET ALL IP ADDRESSES PER DEVICE

In [9]:
# List all IPs from all devices
get_all_ip_addresses = list(nb.ipam.ip_addresses.filter(device=get_all_devices))
#get_all_ip_addresses

GET ALL INTERFACES PER DEVICE

In [10]:
# List all interfaces from all devices
get_all_interfaces = list(nb.dcim.interfaces.filter(device=get_all_devices))
#get_all_interfaces

CREATE TEMPLATE

In [11]:

templates_path = "./templates/"
environment = Environment(loader=FileSystemLoader(templates_path))
template = environment.get_template("dhcpd_static.template")
for device_ip in get_all_ip_addresses:
        for ip_prefix in get_all_ip_prefixes:
                if IPv4Address(device_ip.address.split('/')[0]) in IPv4Network(ip_prefix.prefix):
                        for device_interface in get_all_interfaces:
                                if device_ip.assigned_object.device.name == device_interface.device.name and device_interface.mac_address != None:
                                        content = template.render(
                                        device_name = device_interface.device.name,
                                        mac_address = device_interface.mac_address,
                                        ip_address = device_ip.address.split('/')[0]
                                        )
                                        print(content)


host SRV-PEV-5 {
  hardware ethernet 2A:2F:0A:9F:A8:B5;
  fixed-address 10.30.1.248;
}


In [ ]:
device_type = nb.dcim.device_types.get(slug='cisco_ios').id
device_type

""" devices = nb.dcim.devices.filter(
    role='access_switch',
    site = 'plgn',
    device_type = 'cisco_ios'
) """

sw = nr.filter(
    F(site__slug='plgn') & 
    F(device_role__slug='access_switch') &
    F(device_type__slug='cisco_ios')
    )

result = sw.run(task=napalm_get, getters=['get_interfaces_ip'])
#result = sw.run(napalm_cli,commands=["sh ip int brief"])
#result = sw.run(netmiko_send_command, command_string="show ip int brief")
#print_result(result)
#print(json.dumps(result['SW-PLGN-1'].result, indent=2))
pprint.pprint(result['SW-PLGN-2'].result)
#print_result(result)

In [ ]:
interface = nb.dcim.interfaces.get(name='',device='sw-plgn-1')
interface
""" for i in interfaces:
    print(i) """

Указываем путь к папке с шаблонами

In [ ]:
templates_path = "./templates/"

In [ ]:
def config_switches(task:Task) -> Result:
    # ------------------------------- Configure the interface -------------------------------------------------------------------------------------------------------------------- ----------------
    # Получаем шаблон и отправляем в конфигурацию
    ios_interface_template = task.run(
        name='2 шаг. 1: Получаем шаблон конфигурации интерфейсов',  # mission name
        task=template_file,   # The purpose of the task is to get the configuration template, fixed value
        template='cisco_ios_interface.template',
        path=templates_path
    )

    # Introducing in a specific configuration, configuring the device, pay attention to ".split ('\ n')" Convert configuration to a list
    task.run(
        task=netmiko_send_config,  # The purpose of the task is to complete the configuration, fixed value through Netmiko.
        name='2 шаг. 2: Настройка интерфейсов',
        config_commands=ios_interface_template.result.split('\n'), # Convert template information to a list, then configure it via NetMiko
        cmd_verify=True
    )
    
    # --------------------------------- Configure administrator information ----------------------------
    # Получаем шаблон и отправляем в конфигурацию
    ios_user_template = task.run(
        name='2-1: Получаем шаблон конфигурации пользователей',
        task=template_file,
        template='cisco_ios_user.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='2-2: Настройка пользователей',
        config_commands=ios_user_template.result.split('\n'),
        cmd_verify=True
    ) 
    #-------------DNS-------------
    ios_dns_template = task.run(
        name='3-1: Получаем шаблон конфигурации dns',
        task=template_file,
        template='cisco_ios_dns.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='3-2: Настройка dns',
        config_commands=ios_dns_template.result.split('\n'),
        cmd_verify=True
    ) 
    #-------------NTP-------------
    ios_ntp_template = task.run(
        name='4-1: Получаем шаблон конфигурации ntp',
        task=template_file,
        template='cisco_ios_ntp.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='4-2: Настройка ntp',
        config_commands=ios_ntp_template.result.split('\n'),
        cmd_verify=True
    )
   #-------------LOG-------------
    ios_logging_template = task.run(
        name='5-1: Получаем шаблон конфигурации logging',
        task=template_file,
        template='cisco_ios_logging.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='5-2: Настройка syslog',
        config_commands=ios_logging_template.result.split('\n'),
        cmd_verify=True
    )
    #-------------END-------------
    return Result(
        host=task.host,
        result=f"{task.host} доступен"
    )

In [ ]:
run_result = sw.run(task=config_switches,name='Настройка коммутаторов')
print_result(run_result)